# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 3.0
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 5
Setting new number of workers to: 5
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::114652167878:role/AWSGlueAndS3RoleGrupo2
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 587d5774-2f5a-46f6-8ea3-4003b9c5be2a
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
Waiting for session 587d5774

In [2]:
s3_path = 's3://pfinal-p2-grupo2/raw/userid-profile.tsv'

In [3]:
import pandas as pd

# Crear DynamicFrame a partir del archivo TSV en S3
df = pd.read_csv(s3_path, delimiter="\t")
df

             #id gender   age         country    registered
0    user_000001      m   NaN           Japan  Aug 13, 2006
1    user_000002      f   NaN            Peru  Feb 24, 2006
2    user_000003      m  22.0   United States  Oct 30, 2005
3    user_000004      f   NaN             NaN  Apr 26, 2006
4    user_000005      m   NaN        Bulgaria  Jun 29, 2006
..           ...    ...   ...             ...           ...
987  user_000996      f   NaN   United States  Jul 17, 2006
988  user_000997      m   NaN   United States   Jan 5, 2007
989  user_000998      m   NaN  United Kingdom  Sep 28, 2005
990  user_000999      f   NaN          Poland  Jul 24, 2007
991  user_001000      m   NaN   United States  Mar 24, 2007

[992 rows x 5 columns]


In [4]:
removedCols = ['gender', 'age']
df = df[df.columns.difference(removedCols)]
df.head()

           #id        country    registered
0  user_000001          Japan  Aug 13, 2006
1  user_000002           Peru  Feb 24, 2006
2  user_000003  United States  Oct 30, 2005
3  user_000004            NaN  Apr 26, 2006
4  user_000005       Bulgaria  Jun 29, 2006


In [5]:
df.columns

Index(['#id', 'country', 'registered'], dtype='object')


In [6]:
df.count()

#id           992
country       907
registered    984
dtype: int64


In [7]:
# Borramos filas con valores nulos
df = df.dropna()
df.count()

#id           907
country       907
registered    907
dtype: int64


In [8]:
# conversion a fecha y hora
df['registered'] = pd.to_datetime(df['registered'], format='%b %d, %Y')

In [9]:
df['date'] = df['registered'].dt.strftime("%m%d")
df['registered'] = df['registered'].dt.strftime('%Y-%m-%d')
df.head()

           #id             country  registered  date
0  user_000001               Japan  2006-08-13  0813
1  user_000002                Peru  2006-02-24  0224
2  user_000003       United States  2005-10-30  1030
4  user_000005            Bulgaria  2006-06-29  0629
5  user_000006  Russian Federation  2006-05-18  0518


In [19]:
print(df.loc[df['#id'] == 'user_000224'])

             #id                                country  registered  date
223  user_000224  Congo, the Democratic Republic of the  2006-01-15  0115


In [26]:
if 'Congo, the Democratic Republic of the' in df['country'].values:
    df.loc[df['country'] == 'Congo, the Democratic Republic of the', 'country'] = 'Congo'
    print(df.loc[df['#id'] == 'user_000224'])
if "Korea, Democratic People's Republic of" in df['country'].values:
    df.loc[df['country'] == "Korea, Democratic People's Republic of", 'country'] = 'Democratic Republic of Korea'
    print(df.loc[df['#id'] == 'user_000214'])

             #id                       country  registered  date
213  user_000214  Democratic Republic of Korea  2007-02-04  0204


In [10]:
# ordenamos por fecha ascendente
df = df.sort_values(by='registered', ascending=False)
df.head()

             #id        country  registered  date
526  user_000533          Japan  2009-11-05  1105
490  user_000497  United States  2008-02-16  0216
864  user_000872         Brazil  2008-01-31  0131
644  user_000651  United States  2007-12-17  1217
643  user_000650  United States  2007-12-16  1216


In [11]:
# fecha a estandar ingles sin año 
df.to_csv('s3://pfinal-p2-grupo2/curated/userid-profile.csv', index = False)

In [16]:
s3_dest_path = 's3://pfinal-p2-grupo2/refined/userid-profile/'
df.to_parquet(s3_dest_path, partition_cols=['country'])